In [1]:
import numpy as np
import os
from typing import List, Dict, Optional, Tuple


from tinytorch.core.tensor import Tensor
from tinytorch.core.training import CosineSchedule, clip_grad_norm

# Unit Test - CosineSchedule

In [2]:
def test_unit_cosine_schedule():
    print("🔬 Unit Test: CosineSchedule...")

    # test basic schedule
    schedule = CosineSchedule(max_lr= 0.1, min_lr= 0.01, total_epochs=100)

    # Test start, middle, and end
    lr_start = schedule.get_lr(0)
    lr_middle = schedule.get_lr(50)
    lr_end = schedule.get_lr(100)

    print(f"    Learning rate at epoch 0: {lr_start:.4f}")
    print(f"    Learning rate at epoch 50: {lr_middle:.4f}")
    print(f"    Learning rate at epoch 100: {lr_end:.4f}")

    # Validate behavior
    assert abs(lr_start - 0.1) < 1e-6, f"Expected 0.1 at start, got {lr_start}"
    assert abs(lr_end - 0.01) < 1e-6, f"Expedted 0.01 at end, got {lr_end}"
    assert 0.01 < lr_middle < 0.1, f"Middle LR should be between min and max, got {lr_middle}"

    # Test monotonic decrease in first half
    lr_quarter = schedule.get_lr(25)
    assert lr_quarter > lr_middle, "LR should decrease monotonically in first half"

    print("✅ CosineSchedule works correctly!")

if __name__=='__main__':
    test_unit_cosine_schedule()

🔬 Unit Test: CosineSchedule...
    Learning rate at epoch 0: 0.1000
    Learning rate at epoch 50: 0.0550
    Learning rate at epoch 100: 0.0100
✅ CosineSchedule works correctly!


## Unit Test - Gradient Clipping

In [14]:
def test_unit_clip_grad_norm():
    print("🔬 Unit Test: Gradient Clipping...")

    import sys
    
    param1 = Tensor([1.0, 2.0], requires_grad= True)
    param1.grad = np.array([3.0, 4.0]) # norm = 5.0

    param2 = Tensor([3.0, 4.0], requires_grad= True)
    param2.grad = np.array([6.0, 8.0]) # norm = 10.0

    params = [param1, param2]
    # Total norm = sqrt (25 + 100) = 11.18

    original_norm = clip_grad_norm(params, max_norm=1.0)

    assert original_norm > 1.0, f"Original norm should be > 1.0,  got {original_norm}"

    # check if gradients were clipped
    new_norm = 0.0
    for param in params:
        if isinstance(param.grad, np.ndarray):
            grad_data = param.grad
        else:
            grad_data = param.grad.data

        new_norm += np.sum(grad_data ** 2)
    new_norm = np.sqrt(new_norm)

    print(f"Original norm: {original_norm:.2f}")
    print(f"Clipped norm: {new_norm:.2f}")

    assert abs(new_norm - 1.0) < 1e-6, f"Clipped norm should be 1.0, got {new_norm}"

    ## test small gradients that don;t need clipping
    small_param = Tensor([1.0, 2.0], requires_grad= True)
    small_param.grad = np.array([0.1, 0.2])
    small_params = [small_param]
    original_small = clip_grad_norm(small_params, max_norm= 1.0)
    assert original_small < 1.0, "Small gradients shouldn't be clipped"

    print("✅ Gradient clipping works correctly!")

if __name__=='__main__':
    test_unit_clip_grad_norm()

🔬 Unit Test: Gradient Clipping...
Original norm: 11.18
Clipped norm: 1.00
✅ Gradient clipping works correctly!
